In [31]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler



# Charger les données
data_train = pd.read_csv('data/data_train.csv', delimiter='\t', quotechar='"')

# Préparer les données pour l'imputation
data_not_null = data_train.dropna(subset=['G1', 'G2'])

# Modèle pour imputer G1 en utilisant G2
model_G1 = LinearRegression()
model_G1.fit(data_not_null[['G2']], data_not_null['G1'])

# Modèle pour imputer G2 en utilisant G1
model_G2 = LinearRegression()
model_G2.fit(data_not_null[['G1']], data_not_null['G2'])

# Imputer G1 pour les valeurs manquantes en utilisant G2
missing_G1 = data_train['G1'].isnull() & data_train['G2'].notnull()
data_train.loc[missing_G1, 'G1'] = model_G1.predict(data_train.loc[missing_G1, ['G2']])

# Imputer G2 pour les valeurs manquantes en utilisant G1
missing_G2 = data_train['G2'].isnull() & data_train['G1'].notnull()
data_train.loc[missing_G2, 'G2'] = model_G2.predict(data_train.loc[missing_G2, ['G1']])

# Supprimer les lignes avec des valeurs manquantes restantes
data_train = data_train.dropna(subset=['G1', 'G2'])



# Imputer les valeurs manquantes restantes avec la moyenne
# data_train['G1'].fillna(data_train['G1'].mean(), inplace=True)
# data_train['G2'].fillna(data_train['G2'].mean(), inplace=True)
data_train_cleaned = data_train

# Convertir les variables catégoriques en variables numériques (encodage one-hot)
data_train_encoded = pd.get_dummies(data_train_cleaned, drop_first=True)

In [32]:
data_train_encoded.columns

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

In [33]:
data_train_encoded["address_U"].value_counts()

1    279
0    120
Name: address_U, dtype: int64

In [34]:
display(data_train_encoded.columns)
# Créer une nouvelle colonne pour la consommation totale d'alcool
data_train_encoded['Total_Alcohol_Consumption'] = data_train_encoded['Walc'] + data_train_encoded['Dalc']
data_train_encoded['Combined_G1_G2'] = data_train_encoded['G1'] + data_train_encoded['G2']
data_train_encoded['Combined_Parent_Education'] = data_train_encoded['Medu'] * data_train_encoded['Fedu']
data_train_encoded['Travel_Effect'] = data_train_encoded['address_U'] * data_train_encoded['traveltime']
data_train_encoded['Family_Dynamics'] = data_train_encoded['famsize_LE3'] * data_train_encoded['famrel']
selected_features = [col for col in data_train_encoded.columns if col in ["Combined_G1_G2", 'Combined_Parent_Education', 'Total_Alcohol_Consumption', 'Family_Dynamics','Travel_Effect', 'studytime','failures',"sex_M","school_MS","higher_yes","internet_yes","schoolsup_yes"]]

# Ajouter les nouvelles colonnes encodées manuellement si besoin
X_encoded = data_train_encoded[selected_features]
y_encoded = data_train_encoded['G3']
print(len(X_encoded))
print(len(y_encoded))


Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

399
399


In [35]:
print(X_encoded.shape)
print(y_encoded.shape)


(399, 12)
(399,)


In [36]:
y_encoded.info()

<class 'pandas.core.series.Series'>
Int64Index: 399 entries, 1 to 433
Series name: G3
Non-Null Count  Dtype
--------------  -----
399 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [42]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Définir la grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
}

# Instancier le modèle XGBoost
xgboost_model = xgb.XGBRegressor(random_state=0)

# Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=10, scoring='r2')
grid_search.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres pour XGBoost: {grid_search.best_params_}")

# Utiliser le meilleur modèle
best_xgb_model = grid_search.best_estimator_


Meilleurs hyperparamètres pour XGBoost: {'max_depth': 3, 'n_estimators': 50}


In [43]:
# Les modèles à tester
from math import sqrt

#GridSearchCV ou RandomizedSearchCV. pour trouver les hyper paramétre
models = {
    'Ridge': Ridge(
        alpha=1.0,                 # Paramètre de régularisation
        solver='auto',             # Méthode de résolution
        max_iter=None,             # Nombre maximal d'itérations
        random_state=None,         # Graîne aléatoire
        tol=1e-3                   # Tolérance pour la convergence
    ),
    
    'Lasso': Lasso(
        alpha=1.0,                 # Paramètre de régularisation
        max_iter=1000,             # Nombre maximal d'itérations
        tol=1e-4,                  # Tolérance pour la convergence
        random_state=None          # Graîne aléatoire
    ),
    
    'XGBoost': best_xgb_model,
    
    'LightGBM': lgb.LGBMRegressor(
        boosting_type='gbdt',      # Type de boosting ('gbdt', 'dart', 'goss')
        learning_rate=0.1,         # Taux d'apprentissage
        n_estimators=100,          # Nombre d'arbres
        max_depth=-1,              # Profondeur maximale des arbres (-1 signifie pas de limite)
        subsample=1.0,             # Proportion des données utilisées pour chaque arbre
        colsample_bytree=1.0,      # Proportion de colonnes échantillonnées par arbre
        random_state=None          # Graîne aléatoire
    ),
    
    'Random Forest': RandomForestRegressor(
        n_estimators=100,          # Nombre d'arbres dans la forêt
        max_depth=None,            # Profondeur maximale des arbres
        min_samples_split=2,       # Nombre minimal d'échantillons pour diviser un nœud
        min_samples_leaf=1,        # Nombre minimal d'échantillons dans une feuille
        random_state=None,         # Graîne aléatoire
        n_jobs=-1                  # Utilisation de tous les cœurs disponibles
    ),
    
    'Linear Regression': LinearRegression(
        fit_intercept=True,        # Estimer une ordonnée à l'origine
        n_jobs=None                # Utilisation de tous les cœurs disponibles
    )
}
# Créer le scorer pour la MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}
# Itérer sur les modèles
for model_name, model in models.items():
    # Validation croisée avec R² comme métrique
    cv_scores_r2 = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring='r2')
    mean_r2 = cv_scores_r2.mean()  # Moyenne des scores R²

    # Validation croisée avec MSE comme métrique
    cv_scores_mse = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring=mse_scorer)
    mean_mse = -cv_scores_mse.mean()  # Moyenne de la MSE
    rmse = sqrt(mean_mse)  # Calcul du RMSE

    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 2.0573
  RMSE: 1.4343
  Mean R²: 0.7702
Lasso:
  Mean MSE: 2.0702
  RMSE: 1.4388
  Mean R²: 0.7790
XGBoost:
  Mean MSE: 2.6329
  RMSE: 1.6226
  Mean R²: 0.7119
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000743 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 12
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Diviser les données en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}

# Itérer sur les modèles
for model_name, model in models.items():
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Prédire sur le jeu de test
    y_pred = model.predict(X_test)
    
    # Calculer les métriques
    mean_mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mean_mse)
    mean_r2 = r2_score(y_test, y_pred)
    
    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 0.9548
  RMSE: 0.9771
  Mean R²: 0.8911
Lasso:
  Mean MSE: 0.8804
  RMSE: 0.9383
  Mean R²: 0.8996
XGBoost:
  Mean MSE: 2.7154
  RMSE: 1.6478
  Mean R²: 0.6904
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002489 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 319, number of used features: 12
[LightGBM] [Info] Start training from score 11.824451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with